In [ ]:
# 分成两部分写VGG

In [1]:
import tensorflow as tf 
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import models, Model
from tensorflow import keras
from tensorflow.keras import layers, Sequential, datasets, optimizers
import matplotlib.pyplot as plt
import numpy as np 
import glob
import os
from tqdm import tqdm

In [2]:
import time
import random

In [3]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        exit(-1)

# data

In [5]:
# Global Variables
BATCHSZ = 64
EPOCHS = 100

In [6]:
data_dir= "E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D"
data_root = glob.glob(data_dir + '/*')
for name in glob.glob(data_dir + '/*'):
    print(name)

E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\data.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\data_label.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\test.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\test_label.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\train.npy
E:\Eric_HSI\hyper_data_preprocess\Salinas_w_size_9_num_200_for_2D\train_label.npy


In [7]:
train = np.load(data_root[4])
train_label = np.load(data_root[5])
test = np.load(data_root[2])
test_label = np.load(data_root[3])
train.shape, train_label.shape, test.shape, test_label.shape

((3200, 9, 9, 204), (3200,), (50929, 9, 9, 204), (50929,))

In [8]:
# train_label = tf.keras.utils.to_categorical(train_label)
# test_label = tf.keras.utils.to_categorical(test_label)

In [9]:
class_num = 16
im_height = 9
im_width = 9
im_channel = train.shape[3]
train_num = train.shape[0]
val_num = test.shape[0]

In [10]:
# 根据可用的CPU动态设置并行调用的数量， 应用于 num_parallel_calls
AUTOTUNE = tf.data.experimental.AUTOTUNE

- prefetch(AUTOTUNE)
- 当GPU执行在当前批次执行前向或者后向传播时，我们希望CPU处理下一个批次的数据，以便于数据批次能够迅速被GPU使用。我们希望GPU被完全、时刻用于训练。我们称这种机制为消费者/生产者重叠，消费者是GPU，生产者是CPU。

# dataset顺序：

- 创建实例                             from_tensor_slices                       
- 重组（较大的buffer_size）             shuffle
- 重复                                  repeat
- 数据预处理、数据扩增，使用多线程等                  map
- 批次化                                batch
- 预取数据                             prefectch

In [12]:
# load train dataset
train_db = tf.data.Dataset.from_tensor_slices((train, train_label))
# train_dataset = train_dataset.shuffle(buffer_size=train_num).repeat().batch(BATCHSZ).prefetch(AUTOTUNE)
train_db= train_db.shuffle(buffer_size=train_num).batch(BATCHSZ).prefetch(AUTOTUNE)

# load test dataset
test_db = tf.data.Dataset.from_tensor_slices((test, test_label))
# val_dataset = val_dataset.repeat().batch(BATCHSZ).prefetch(AUTOTUNE)
test_db = test_db.batch(BATCHSZ).prefetch(AUTOTUNE)

In [13]:
train_db, test_db 

(<PrefetchDataset shapes: ((None, 9, 9, 204), (None,)), types: (tf.float32, tf.int32)>,
 <PrefetchDataset shapes: ((None, 9, 9, 204), (None,)), types: (tf.float32, tf.int32)>)

# model

In [14]:
conv_layers = [  # 5 units of conv + max pooling
    # unit 1
    layers.Conv2D(64, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.Conv2D(64, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=[2, 2], strides=2, padding='same'),

    # unit 2
    layers.Conv2D(128, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.Conv2D(128, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=[2, 2], strides=2, padding='same'),

    # unit 3
    layers.Conv2D(256, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.Conv2D(256, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=[2, 2], strides=2, padding='same'),

    # unit 4
    layers.Conv2D(512, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.Conv2D(512, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=[2, 2], strides=2, padding='same'),
    
    # unit 5
    layers.Conv2D(512, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.Conv2D(512, kernel_size=[3, 3], padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=[2, 2], strides=2, padding='same')
]

In [15]:
# 一个网络分成两部分来写，也可以部分成两部分，用 flatten 层，这里写成两部分，加深对编写网络层的理解
# 第一部分，一个Sequential
conv_net = Sequential(conv_layers)
# x = tf.random.normal([4, 32, 32, 204])
# out = conv_net(x)
# # 从这里可以看到展平后的维度
# print(out.shape)  # (4, 1, 1, 512)

In [16]:
# 第一部分，另一个Sequential
fc_net = Sequential([
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(16, activation='relu'),
])

# 网络定义完成
conv_net.build(input_shape=[None, 9, 9, 204])
fc_net.build(input_shape=[None, 512])

# 现在的 loss 非常小， 因为这个问题比较复杂
optimizer = optimizers.Adam(lr=1e-4)

# 训练

In [17]:
# 两个结构的变量组合， 同时求导
# [1, 2] + [3, 4] = [1, 2, 3, 4]
variables = conv_net.trainable_variables + fc_net.trainable_variables

In [24]:
for epoch in tqdm(range(20)):
    for step, (x, y) in enumerate(train_db):
        with tf.GradientTape() as tape:
            # [b, 9, 9, 204] -> [b, 1, 1, 512]
            out = conv_net(x)
            out = tf.reshape(out, [-1, 512])
            # [b, 512] -> [b, 16]
            logits = fc_net(out)
            y_onehot = tf.one_hot(y, depth=16)
            loss = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)
            loss = tf.reduce_mean(loss)

        # 反向传播
        grads = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(grads, variables))

    if step % 100 == 0:
        print(epoch, step, 'loss', float(loss))

    total_num = 0
    total_correct = 0

    # 再哪里进行测试，要自己把握，测试的时间影像训练效率
    for x, y in test_db:
        out = conv_net(x)
        out = tf.reshape(out, [-1, 512])
        logtis = fc_net(out)
        prob = tf.nn.softmax(logtis, axis=1)
        pred = tf.argmax(prob, axis=1)

        # int64 -> int32
        pred = tf.cast(pred, dtype=tf.int32)
        # booler -> int
        correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
        correct = tf.reduce_sum(correct)

        total_num += x.shape[0]
        total_correct += int(correct)

    acc = total_correct / total_num
    print(epoch, 'acc', acc)

100%|██████████| 20/20 [02:27<00:00,  7.38s/it]19 acc 0.6436411474798248

